In [1]:
from __future__ import division
import sys,os
import time
import argparse

import numpy as np
import matplotlib.pyplot as plt
from   matplotlib import cm

import pandas as pd
from   glob import glob

## Import MKID libraries

In [2]:
import ResonanceFitter as fitres
import ResonanceFitResult as fitclass

sys.path.append('/home/nexus-admin/NEXUS_RF/AcquisitionScripts')
from VNAMeas import *

import PyMKID_USRP_functions as puf

ModuleNotFoundError: No module named 'VNAMeas'

## Runtime options

In [3]:
## Flag to display plots
show_plots = False

## Which powers to look at
pwr_min = -55
pwr_max = -15
pwr_stp =   5

## What's the temperature range
T_min =  10 
T_max = 350

## Set up global plot options

In [4]:
## Set up matplotlib options for plots
plt.rcParams['axes.grid'] = True
plt.rcParams.update({'font.size': 12})
#plt.rc('text', usetex=True)
plt.rc('font', family='serif')
dfc = plt.rcParams['axes.prop_cycle'].by_key()['color']

## Temperature color scale
norm_T = plt.Normalize(vmin=T_min,vmax=T_max)
norm_P = plt.Normalize(vmin=pwr_min,vmax=pwr_max)

## Paths & file handling

In [5]:
## Series identifier
day    = '20220419'

## Path to VNA data
dataPath = '/data/TempSweeps/VNA/'

## Create a place to store processed output
# out_path = '/data/ProcessedOutputs/out_' + series

## Method to pull all files for a given RF power

In [8]:
def get_input_files(day, pwr):

    ## Get all folders in date
    datePath    = os.path.join(dataPath, day)

    ## Define the series path from the series
    srPath = os.path.join(datePath, day+"_*")

    ## File string format
    fn_form = ("TPsweep_T*_P%i" % pwr) + "_*.h5"

    ## Find and sort the relevant directories in the series
    print("Searching for files in:", srPath)
    print(" with form:", fn_form)
    vna_file_list = glob(os.path.join(srPath,fn_prefix+fn_suffix))
    vna_file_list.sort(key=os.path.getmtime)
    print("Using files:")
    for fname in vna_file_list:
        print("-",fname)
    return vna_file_list

## Method to fit the spectra for each power

In [9]:
def fit_single_file(file_name):
    # raw_f, raw_VNA, amplitude = puf.read_vna(file_name)
    #power = -14 + 20*np.log10(amplitude)

    ## Open the h5 file for this power and extract the class
    sweep = decode_hdf5(file_name)
    # sweep.show()

    ## Extract the RF power from the h5 file
    print("Extracting data for power:",sweep.power,"dBm")
    # power_list.append(sweep.power)

    ## Parse the file, get a complex S21 and frequency in GHz
    f = sweep.frequencies / 1.0e9
    z = sweep.S21realvals + 1j*sweep.S21imagvals

    ## Create an instance of a file fit result class
    this_f_r = fitclass.SingleFileResult(file_name)
    this_f_r.power = sweep.power
    this_f_r.start_T = sweep.start_T
    this_f_r.final_T = sweep.final_T

    ## Fit this data file
    fr, Qr, Qc, Qi, fig = fitres.sweep_fit(f,z,this_f_r,start_f=f[0],stop_f=f[-1])

    if (len(fr) > 1):
        fr = fr[0]
        Qr = Qr[0]
        Qc = Qc[0]
        Qi = Qi[0]

    ## Show the results of the fit
    this_f_r.show_fit_results()

    ## Get the color for this spectrum
    temp = file_name.split('/')[-1].split('_')[1][1:]
    color = cm.jet(norm(float(temp)))
    
    ax_main.plot(f,20*np.log10(abs(np.sqrt(z*z))),label=temp+' mK',color=color, alpha=0.25)

    ## Save the figure
    plt.gcf()
    plt.title("Power: "+str(sweep.power)+" dBm, Temperature: "+str(np.mean(sweep.start_T))+" mK")
    fig.savefig(os.path.join(out_path,"freq_fit_P"+str(sweep.power)+"dBm.png"), format='png')

    ## Return the fit parameters
    return sweep.power, fr, Qr, Qc, Qi, this_f_r, temp

## Outer Loop
Loop over each RF stimulus power, get the S21 vs F data for each T, fit to MB equations, store fit data. Need to separate out Al vs Nb 7 resonators.

In [11]:
## Define the set of powers we're interested in
power_range = np.arange(start=pwr_min, stop=pwr_max+pwr_stp, step=pwr_stp)
n_powers    = len(power_range)

## Start the loop over each power
for i in np.arange(n_powers):
    pwr = power_range[i]
    
    ## Get the list of files at this power
    p_files = get_input_files(day, pwr)
    print("Power = ",pwr," dBM files:")
    for file in p_files:
        print("\t",file)


[-55 -50 -45 -40 -35 -30 -25 -20 -15]


## Money plot
Frequency shift vs temperature, one line per RF power.

In [ ]:
## Create a figure for the spectra
fig_main = plt.figure(200,figsize=(8,6))
ax_main  = fig_main.gca()
plt.title('MKID Temperature Sweep: Al', fontdict = {'fontsize': 18})
plt.xlabel(r'Temperature [mK]', fontdict = {'fontsize': 18})
plt.ylabel(r'$\Delta f$ [kHz]', fontdict = {'fontsize': 18})
cbar=plt.colorbar(cm.ScalarMappable(cmap=cm.jet, norm=norm_P),shrink=0.8)
cbar.set_label('RF Power [dBm]', size=16)
plt.legend(loc='best')#,bbox_to_anchor=(1.,0.5))